In [1]:
import pandas as pd

data = pd.read_csv("/home/sslunder24/project/moral/delphi/dataset/commonsense-norm-bank/freeform/test_freeform.tsv",delimiter='\t')

In [100]:
# Iterate over each line in the 'text_label' column of data
unique_lines = {}
kolist = []

for idx, line in enumerate(data['text_label']):
    if line not in unique_lines:
        unique_lines[line] = 1
    else:
        unique_lines[line] = unique_lines[line]+1


In [104]:
#for idx, line in enumerate(data['text_label']):
#    if line=='도덕이야.':
#        data.at[idx, 'text_label'] = "도덕적입니다."


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("Krittapad/nllb-200-distilled-1.3B-peft-th-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Krittapad/nllb-200-distilled-1.3B-peft-th-en-de").to(device)


def translate(text):
    inputs = tokenizer.encode(text, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(inputs)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Initialize dictionary to store unique lines with their row numbers
unique_lines = {}
kolist = []

# Iterate over each line in the 'text_label' column of data
for idx, line in enumerate(data['text_label']):
    if line not in unique_lines:
        unique_lines[line] = [idx]
    else:
        unique_lines[line].append(idx)

# Translate unique lines
translated_lines = {}
for line in tqdm(unique_lines.keys()):
    translated_lines[line] = translate(line)

# Append index number pair with translated line
for line, indices in unique_lines.items():
    for idx in indices:
        kolist.append((idx, translated_lines[line]))

# Update the 'text_label' column with translated text
for idx, translation in kolist:
    data.at[idx, 'text_label'] = translation

# Example output


  2%|█▌                                                                                                   | 25/1566 [00:02<02:44,  9.40it/s]


KeyboardInterrupt: 

In [ ]:
# Update the 'text_label' column with translated text
for idx in tqdm(data.index, desc="Translating"):
    translated_text = translate(data.at[idx, 'input_sequence'])
    data.at[idx, 'input_sequence'] = translated_text

Translating:   0%|                                                                                     | 58/95780 [00:12<5:17:44,  5.02it/s]

In [2]:
data.to_csv("/home/sslunder24/project/moral/delphi/dataset/commonsense-norm-bank/freeform/test_freeform_de.tsv",sep='\t')